# Dropout Mask

In [ ]:
import numpy as np
import sys

np.random.seed(1)

images = np.loadtxt('data/train_100.csv', delimiter=',')
labels = np.loadtxt('data/train_100_labels.csv', delimiter=',')
test_labels = np.loadtxt('data/test_100_labels.csv', delimiter=',')
test_images = np.loadtxt('data/test_100.csv', delimiter=',')

def relu(x):
    return (x > 0) * x

def relu2deriv(output):
    return output > 0

alpha, iterations, hidden_size = (0.005, 300, 100)
pixels_per_image, num_labels = (784, 10)

weights_0_1 = 0.2 * np.random.random(pixels_per_image, hidden_size) - 0.1
weights_0_2 = 0.2 * np.random.random(hidden_size, num_labels) - 0.1

for j in range(iterations):
    error, correct_cnt = (0.0, 0)
    for i in range(len(images)):
        layer_0 = images[i:i+1]
        layer_1 = relu(np.dot(layer_0, weights_0_1))
        dropout_mask = np.random.randint(2, size=layer_1.shape)
        layer_1 *= dropout_mask * 2
        layer_2 = np.dot(layer_1, weights_0_2)

        error += np.sum(labels[i:i+1] - layer_2) ** 2
        correct_cnt += int(np.argmax(layer_2) == np.argmax(labels[i:i+1]))

        layer_2_delta = (labels[i:i+1] - layer_2)
        layer_1_delta = layer_2_delta.dot(weights_0_2.T) * relu2deriv(layer_1)
        layer_1_delta *= dropout_mask

        weights_1_2 = alpha * layer_1.T.dot(layer_2_delta)
        weights_0_1 += alpha * layer_0.T.dot(layer_1_delta)

    if(j % 10 == 0):
        test_error = 0.0
        test_correct_cnt = 0

        for i in range(len(test_images)):
            layer_0 = test_images[i:i+1]
            layer_1 = relu(np.dot(layer_0, weights_0_1))
            layer_2 = np.dot(layer_1, weights_0_2)

            test_error += np.sum((test_labels[i:i+1] - layer_2) ** 2)
            test_correct_cnt += int(np.argmax(layer_2) == np.argmax(test_labels[i:i+1]))

        
        sys.stdout.write("\n" + 
                         "I:" + str(j) + \
                         " Test-Err:" + str(test_error/float(len(test_images)))[0:5] + \
                         " Test-Acc:" + str(test_correct_cnt/float(len(test_images)))[0:5] + \
                         " Train-Err:" + str(error/float(len(images)))[0:5] + \
                         " Train-Acc:" + str(correct_cnt/float(len(images)))[0:5])